In [1]:
import pandas as pd

In [ ]:
john_review = pd.read_csv("john_ca_plates_with_valid.csv", index_col="index", encoding='ISO-8859-1')
jiahao_review = pd.read_csv("jiahao_review_ca.csv", index_col="index")
cameron_review_1 = pd.read_csv("cameron_first_500.csv", index_col="index")
cameron_review_2 = pd.read_csv("cameron_extra_600.csv", index_col="index")

,valid,plate,customer_meaning,reviewer_comments,status,review_reason_code
index,,,,,,
22979,3,TYJUICY,THANK YOU TO MY DECEASED HUSBAND,PULLED FOR JUICY,N,1
8415,3,DUB JUKE,my nickname is dub. I own a nissan juke car.,dub can have a drug connotation,N,3
14100,0,PMD 14,MY SONS INITIALS AND HIS ATHLETIC NUMBER,14 = GANG NUMBER,N,2
2781,0,415M3,CALL CODE FOR WORK,415 AREA CODE,N,2
14733,3,DR DABS,NICKNAME,"DABS=BUTANE HASH OIL, DRUG CONNOTATION",N,7D


In [46]:
def meaningFromCode(row):
    valid = row["valid"]
    if valid == 1:
        return row["customer_meaning"]
    if valid == 2:
        return row["reviewer_comments"]
    if valid == 3:
        return row["customer_meaning"] + " " + row["reviewer_comments"]
    return ""

In [72]:
# Prepare John's Reviewed plates
john_review = john_review[john_review["valid"].notna()]
john_review_prepped = john_review.copy()
john_review_prepped = john_review_prepped[john_review["valid"].notna()]
john_review_prepped["valid"] = john_review_prepped["valid"].astype(int)
john_review_prepped = john_review_prepped[john_review_prepped["valid"] != 0]
john_review_prepped["meaning"] = john_review_prepped.apply(meaningFromCode, axis=1)

In [63]:
# Prepare Cameron's Reviewed plates
cameron_review = pd.concat([cameron_review_1, cameron_review_2])
cameron_review_prepped = cameron_review.copy()
cameron_review_prepped = cameron_review_prepped[cameron_review_prepped["valid"].notna()]
cameron_review_prepped["valid"] = cameron_review_prepped["valid"].astype(int)
cameron_review_prepped = cameron_review_prepped[cameron_review_prepped["valid"] != 0]
cameron_review_prepped["meaning"] = cameron_review_prepped.apply(meaningFromCode, axis=1)

In [69]:
# Prepare Jiahao's Reviewed plates
jiahao_review_prepped = jiahao_review.copy()
jiahao_review_prepped["valid"] = jiahao_review_prepped["review_reason_code.1"]
jiahao_review_prepped = jiahao_review_prepped.drop("review_reason_code.1", axis="columns")

jiahao_review_prepped = jiahao_review_prepped[jiahao_review_prepped["valid"].notna()]
jiahao_review_prepped["valid"] = jiahao_review_prepped["valid"].astype(int)
jiahao_review_prepped = jiahao_review_prepped[jiahao_review_prepped["valid"] != 0]

In [74]:
final_prepped_df = pd.concat([john_review_prepped, cameron_review_prepped, jiahao_review_prepped])
total_reviewed_plates = len(cameron_review) + len(john_review) + len(jiahao_review)
total_valid_plates = len(final_prepped_df)
print(f"Total reviewed records: {total_reviewed_plates}")
print(f"Total removed records: {total_reviewed_plates - total_valid_plates}")
print(f"Total records with valid plate meanings: {total_valid_plates}")

Total reviewed records: 1998
Total removed records: 528
Total records with valid plate meanings: 1470


In [75]:
final_prepped_df.to_csv("combined_reviewed_ca_plates.csv", index=True, index_label="index")